# Importing Libraries

In [35]:
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
#from keras.models import Model
from tensorflow.keras.models import Model
#from tensorflow.python.keras.layers import Input
from keras.layers import Input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [36]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

In [37]:
from operator import truediv

In [38]:
from plotly.offline import init_notebook_mode

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral

init_notebook_mode(connected=True)
%matplotlib inline

# Data Loading

In [40]:
Dataset = 'IP'
test_ratio = 0.7
windowSize = 25

In [41]:
def loadData(name):
    data_path = os.path.join(os.getcwd(),'data')
    if name == 'IP':
        data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(os.path.join(data_path, 'Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(os.path.join(data_path, 'PaviaU.mat'))['PaviaU']
        labels = sio.loadmat(os.path.join(data_path, 'PaviaU_gt.mat'))['PaviaU_gt']
    return data, labels

In [42]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState, stratify=y)
    return X_train, X_test, y_train, y_test

In [43]:
def applyPCA(X, numComponents = 75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], numComponents))
    return newX, pca

In [44]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2* margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [45]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels =True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin +1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [46]:
X, y = loadData(Dataset)

X.shape, y.shape

((145, 145, 200), (145, 145))

In [47]:
K = X.shape[2]

In [48]:
K = 30 if Dataset == 'IP' else 15
X,pca = applyPCA(X, numComponents=K)

X.shape

(145, 145, 30)

In [49]:
X, y = createImageCubes(X, y, windowSize=windowSize)

X.shape, y.shape

((10249, 25, 25, 30), (10249,))

In [50]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((3074, 25, 25, 30), (7175, 25, 25, 30), (3074,), (7175,))

# Model and Training

In [51]:
Xtrain = Xtrain.reshape(-1, windowSize, windowSize, K, 1)
Xtrain.shape

(3074, 25, 25, 30, 1)

In [52]:
ytrain = np_utils.to_categorical(ytrain)
ytrain.shape

(3074, 16)

In [53]:
S = windowSize
L = K
output_units = 9 if (Dataset == 'PU' or Dataset == 'PC') else 16

In [56]:
## Input Layer
input_layer = Input((S, S, L, 1))

## Convolutional Layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3 ,7), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 5), activation='relu')(conv_layer1)
conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(conv_layer2)
print(conv_layer3.shape)
conv3d_shape = conv_layer3.shape
conv_layer3 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layer3)
conv_layer4 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv_layer3)
 
flatten_layer = Flatten()(conv_layer4)

## fully connected layers
dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=output_units, activation='softmax')(dense_layer2)

(None, 19, 19, 18, 32)


In [57]:
# define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

In [58]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 5,122,176
Trainable params: 5,122,176
Non-trainable params: 0
_________________________________________________________________


In [61]:
#compiling the model
adam = Adam(learning_rate=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
# checkpoint
filepath = "best-model.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(x=Xtrain, y=ytrain, batch_size=256, epochs=100, callbacks=callbacks_list)

Epoch 1/100
13/13 [==============================] - 229s 17s/step - loss: 2.7340 - accuracy: 0.1344
Epoch 2/100
13/13 [==============================] - 228s 17s/step - loss: 2.4055 - accuracy: 0.1831
Epoch 3/100
13/13 [==============================] - 223s 17s/step - loss: 2.2084 - accuracy: 0.3159
Epoch 4/100
13/13 [==============================] - 212s 16s/step - loss: 1.8963 - accuracy: 0.3393
Epoch 5/100
13/13 [==============================] - 160s 12s/step - loss: 1.6531 - accuracy: 0.3910
Epoch 6/100
13/13 [==============================] - 166s 13s/step - loss: 1.2478 - accuracy: 0.5556
Epoch 7/100
13/13 [==============================] - 165s 13s/step - loss: 0.7462 - accuracy: 0.7674
Epoch 8/100
13/13 [==============================] - 169s 13s/step - loss: 0.7098 - accuracy: 0.7775
Epoch 9/100
13/13 [==============================] - 168s 13s/step - loss: 0.4006 - accuracy: 0.8774
Epoch 10/100
13/13 [==============================] - 171s 13s/step - loss: 0.1679 - accura

# Validation

In [ ]:
# load best weights
model.load_weights("best-model.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
Xtest = Xtest.reshape(-1, windowSize, windowSize, K, 1)
Xtest.shape

In [ ]:
ytest = np_utils.to.categorical(ytest)
ytest.shape

In [ ]:
Y_pred_test = model.predict(Xtest)
y_pred_test = np.argmax(Y_pred_test, axis=1)

classification  =classification_report(np.argmax(ytest, axis=1), y_pred_test)
print(classification)

In [ ]:
def AA_andEachClassAcCuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np,diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [ ]:
def reports (X_test,y_test,name):
    #start = time.time()
    Y_pred = medel.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)
    #end = time.time()
    #print(end - start)
    if name == 'IP':
        target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn', 'Grass-trees', 'Grass-pature-mowed', 'Hay-windrowed',
                        'Oats', 'Soyabean-notill', 'Soybean-mintill', 'Soyabean-clean', 'Wheat', 'Woods', 'Buildings=Grass-Tress-Drives',
                         'Stone-Stell-Towers']
    elif name == 'SA':
        target_names = ['Brocoli_green_weds_1', 'Brocoli_green_weds_2', 'Fallow', 'Fallow_rough_plow', 'Falloww_smooth', 'Stubble', 
                        'Celery', 'Grapes_untrained', 'Soil_vinyard_develop', 'Corn_senesced_green_weds', 'Lettuce_roamine_4wk',
                        'Lettuce_romaine_5wk', 'Lettuce_romaine_6wk', 'Lettuce_romaine_7wk', 'Vinyard_vertical_trellis']
    elif name == 'PU':
        target_names = ['Asphalt', 'Meadows', 'Gravel', 'Trees', 'Painted metal sheets', 'Bare Soil', 'Bitumen', 'Self-Blocking-Bricks', 
                        'Shadows']
    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(y_test, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=32)
    Test_Loss = score[0]*100
    Test_accuracy  = score[1]*100

    return classification, confusion, Test_Loss, Test_accuracy, oa*100, each_acc*100, aa*100, kappa*100

In [ ]:
classification, confusion, Test_Loss, Test_accuracy, oa, each_acc, aa, kappa = reports(Xtest, ytest, Dataset)
classification = str(classification)
confusion = str(confusion)
file_name = "classification_report.txt"

with open(file_name, 'w') as x_file:
    x_file.write('{} Test loss(%)'.format(Test_Loss))
    x_file.write('\n')
    x_file.write('{} Test accuracy(%)'.format(Test_Accuracy))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{} Kappa accuracy(%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy(%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy(%)'.format(aa))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}'.format(classification))
    x_file.write('\n')
    x_file.write('{}'.format(confusion))

In [ ]:
def Patch(data,height_index,width_index):
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice  = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]
    
    return patch

In [ ]:
# load original image
X, y = loaddata(Dataset)

In [ ]:
height = y.shape[0]
width = y.shape[1]
PATCH_SIZE = windowSize
numComponents = K

In [ ]:
X, pca = applyPCA(X, numComponents=numComponents)

In [ ]:
X = padWithZeros(X, PATCH_SIZE//2)

In [ ]:
# calculate the predicted image
outputs = np.zeros(height,width))
for i in range(height):
    for j in range(width):
        target = int(y[i,j])
        if target == 0:
            continue
        else:
            image_patch=Patch(X,i,j)
            X_test_image = image_patch.reshape(1,image_patch.shape[0], image_patch.shape[1], image_patch.shape[2], 1).astype('float32')
            prediction = (model.predict(X_test_image))
            outputs[i][j] = prediction+1

In [ ]:
ground_truth = spectral.imshow(classes = y, figsize =(7,7))

In [ ]:
predict_image = spectral.imshow(classes = outputs.astype(int),figsize =(7,7))

In [ ]:
spectral.save_rgb("predictions.jpg", outputs.astype(int), colors=spectral.spy_colors)